# Viszualizing Distances on a world map #

The information about the locations can be found in the folder "data". For the aim of the project, we differntiate between datacenters (hosts) and users (clients). They're saved in csv-files containing the main information. 

In [1]:
import pandas as pd
import numpy as np
import folium
import folium.plugins
import geo_convert
from shapely.geometry import LineString
import geopandas as gpd
from geopy.distance import geodesic

The data is imported into pandas dataframe.

In [2]:
hosts_df = pd.read_csv("data/hosts.csv", dtype={"Name": str, "land": str, "street": str, "number": str, "zip": str, "city": str})
clients_df = pd.read_csv("data/clients.csv", dtype={"Name": str, "land": str, "street": str, "number": str, "zip": str, "city": str})
def get_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.DataFrame(df)
    if "longitude" not in list(df):
        coords = []
        for _,v in df.iterrows():
            address = {"street": v["street"],
                   "number": v["number"],
                   "city": v["city"],
                   "country": v["land"],
                   "zip_code": v["zip"]}
            coords.append(geo_convert.geocode_address(address))
        df["latitude"] = pd.Series([i[0] for i in coords])
        df["longitude"] = pd.Series([i[1] for i in coords])
    return df

clients_df = get_coordinates(clients_df)
clients_df.to_csv("data/clients.csv", index=False, encoding="utf8")
hosts_df = get_coordinates(hosts_df)
hosts_df.to_csv("data/hosts.csv", index=False, encoding="utf8")
print(clients_df)
print(hosts_df)

   client_id                 name     land                       street  \
0          1        Suomen Pankki  Finland               Snellmaninkatu   
1          2     Banque de France   France  rue Croix des Petits-Champs   
2          3      Banco de Espana    Spain                    C/ Alcalá   
3          4  Τράπεζα Της Ελλάδος   Greece                El. Venizelou   

  number    zip      city           notes   latitude  longitude  
0      2    170  Helsinki   Finlands bank  60.170329  24.953266  
1     31  75001     Paris  Bank of France  48.863840   2.339779  
2     48  28014    Madrid   Bank of Spain  40.298463  -3.301920  
3     21  10250    Athina  Bank of Greece  37.984860  23.755946  
   host_id name     land           street number    zip       city  \
0        1  EZB  Germany  Sonnemannstraße     22  60314  Frankfurt   

               notes  latitude  longitude  
0  Adress of the EZB  50.11133   8.704547  


We have now imported the data and looked up the geo-data. In the folowing part, the informations will be visualized on a map.

In [5]:
map_folium = folium.Map(location=[48.8566969, 2.3514616], zoom_start=2)
connections = pd.read_csv("data/connections.csv", encoding="utf8")
for i, v in hosts_df.iterrows():
    folium.Marker([v["latitude"], v["longitude"]],
                  popup="<h3>%s</h3>%s" % (v["name"], v["notes"]),
                  icon = folium.Icon(color="lightgray", icon="cloud", prefix="fa")).add_to(map_folium)
lines = []
for i, v in clients_df.iterrows():
    folium.Marker([v["latitude"], v["longitude"]],
                  popup="<h3>%s</h3>%s" % (v["name"], v["notes"]),
                  icon = folium.Icon(color="red", icon="user", prefix="fa")).add_to(map_folium)
    for _, h in connections.loc[connections.client_id == v["client_id"]].iterrows():
        for _, host in hosts_df.loc[hosts_df.host_id == h["host_id"]].iterrows():
            lines.append((LineString([(v["longitude"], v["latitude"]), 
                                    (host["longitude"], host["latitude"])]),
                          np.round(geodesic((v["latitude"], v["longitude"]),
                                   (host["latitude"], host["longitude"])).km, 2)))

folium.plugins.Fullscreen().add_to(map_folium)
line_gdf = gpd.GeoDataFrame({'geometry': [l[0] for l in lines], 'distance': [l[1] for l in lines]})
# Zur Berechnung der Entfernung:
# >>> helsinki = (60.170329, 24.9532664)
# >>> frankfurt = (50.1113302, 8.7045469)
# >>> geodesic(helsinki, frankfurt).km
# 1517.5562305192686


for _, r in line_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    distance = r['distance']
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup("Distance: %s km" % distance).add_to(geo_j)
    geo_j.add_to(map_folium)
map_folium.save("export/map_visualized.html")
map_folium